# Creation of features
## Repeating certain data pre-processing of the previous file as it was processed in a different environment

## Preprocessing as done in the previous file

In [1]:
import numpy as np
import pandas as pd
# import bcolz
import seaborn as sns
import os
import matplotlib.pyplot as plt

%matplotlib inline
data_path = "C:/Users/sp14657/all/"
seed=1204
# def save_array(fname, arr):
#     """
#     save numpy array to file
#     """
#     c=bcolz.carray(arr, rootdir=fname, mode='w')
#     c.flush()

# def load_array(fname):
#     return bcolz.open(fname)[:]

def get_submission(test,item_cnt_month):
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path + 'submission.csv',index=False)
    return sub

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder



In [2]:
from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle

In [3]:
transactions    = pd.read_csv(os.path.join(data_path, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(data_path, 'items.csv'))
item_categories = pd.read_csv(os.path.join(data_path, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(data_path, 'shops.csv'))
test            = pd.read_csv(os.path.join(data_path, 'test.csv.gz'))

In [4]:
transactions = transactions[transactions.item_price<100000]
transactions = transactions[transactions.item_cnt_day<1001]

In [5]:
median = transactions[(transactions.shop_id==32)&(transactions.item_id==2973)&(transactions.date_block_num==4)&(transactions.item_price>0)].item_price.median()
transactions.loc[transactions.item_price<0, 'item_price'] = median

In [6]:
# Якутск Орджоникидзе, 56
transactions.loc[transactions.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
transactions.loc[transactions.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
transactions.loc[transactions.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [7]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in transactions['date_block_num'].unique():
    cur_shops = transactions.loc[transactions['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = transactions.loc[transactions['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates to get rid of duplicates
gb = transactions.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

C:\Users\sp14657\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [8]:
# Fix column names
print(gb.columns.values)
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
print(gb.columns.values)

[('shop_id', '') ('item_id', '') ('date_block_num', '')
 ('item_cnt_day', 'target')]
['shop_id' 'item_id' 'date_block_num' 'target']


In [9]:
print(gb.columns.values)

['shop_id' 'item_id' 'date_block_num' 'target']


In [10]:
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

In [11]:
# merge with category id
all_data =pd.merge(all_data,items,on=['item_id'],how='left')

In [12]:
#shop-month aggregates
gb = transactions.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'shop_block_target_sum':'sum','shop_block_target_mean':np.mean}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

C:\Users\sp14657\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [13]:
#item-month aggregates
gb = transactions.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'item_block_target_sum':'sum','item_block_target_mean':np.mean}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

C:\Users\sp14657\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [14]:
#item category-month aggregates
transactions =pd.merge(transactions,items,on=['item_id'],how='left')
gb = transactions.groupby(['item_category_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'item_cat_block_target_sum':'sum','item_cat_block_target_mean':np.mean}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_category_id', 'date_block_num']).fillna(0)

C:\Users\sp14657\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [15]:
def downcast_dtypes(df):
    '''
    Changes column types in the dataframe: 

    `float64` type to `float32`
    `int64`   type to `int32`
    '''

    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]

    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)

    return df


In [16]:
downcast_dtypes(all_data)

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,item_cat_block_target_sum,item_cat_block_target_mean
0,59,22154,0,1.0,ЯВЛЕНИЕ 2012 (BD),37,2017.0,1.092041,18.0,1.000000,6094.0,1.053414
1,59,2552,0,0.0,DEEP PURPLE The House Of Blue Light LP,58,2017.0,1.092041,0.0,0.000000,287.0,0.969595
2,59,2554,0,0.0,DEEP PURPLE Who Do You Think We Are LP,58,2017.0,1.092041,1.0,1.000000,287.0,0.969595
3,59,2555,0,0.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,2017.0,1.092041,2.0,1.000000,268.0,1.007519
4,59,2564,0,0.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,2017.0,1.092041,5.0,1.000000,701.0,0.990113
5,59,2565,0,0.0,DEEP PURPLE Stormbringer (фирм.),56,2017.0,1.092041,2.0,1.000000,268.0,1.007519
6,59,2572,0,0.0,DEFTONES Koi No Yokan,55,2017.0,1.092041,12.0,1.000000,12527.0,1.025878
7,59,2573,0,0.0,DEL REY LANA Born To Die,55,2017.0,1.092041,16.0,1.142857,12527.0,1.025878
8,59,2574,0,2.0,DEL REY LANA Born To Die The Paradise Editio...,55,2017.0,1.092041,119.0,1.144231,12527.0,1.025878
9,59,2593,0,0.0,DEPECHE MODE Music For The Masses,55,2017.0,1.092041,5.0,1.000000,12527.0,1.025878


In [17]:
all_data.to_csv(data_path+'new_sales_v1_reduced.csv',index=False)

In [18]:
all_data.shape

(10913804, 12)

# Calculating the lag features

In [19]:
index_cols = ['shop_id', 'item_id', 'date_block_num','item_category_id']
cols_to_rename = list(all_data.columns.difference(index_cols))
for i in ['item_name']:
    cols_to_rename.remove(i)
print(cols_to_rename)

['item_block_target_mean', 'item_block_target_sum', 'item_cat_block_target_mean', 'item_cat_block_target_sum', 'shop_block_target_mean', 'shop_block_target_sum', 'target']


In [20]:
cols_gb_item = [i for i in cols_to_rename if 'item_block' in i]
cols_gb_shop = [i for i in cols_to_rename if 'shop_block' in i]
cols_gb_cat = [i for i in cols_to_rename if 'item_cat' in i]
cols_gb_all = ['target']
cols_gb_key=[['item_id'],['shop_id'],['item_category_id'],['shop_id','item_id']]
cols_gb_value = [cols_gb_item,cols_gb_shop,cols_gb_cat,cols_gb_all]
print(cols_gb_value)

[['item_block_target_mean', 'item_block_target_sum'], ['shop_block_target_mean', 'shop_block_target_sum'], ['item_cat_block_target_mean', 'item_cat_block_target_sum'], ['target']]


In [21]:
shift_range = [1,2,3,5,12]
for month_shift in shift_range:
    for k,v in zip(cols_gb_key,cols_gb_value): 
        index_col = ['date_block_num'] + k
        train_shift = all_data[index_col + v].copy().drop_duplicates()

        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift

        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in v else x
        train_shift = train_shift.rename(columns=foo)
        all_data = pd.merge(all_data, train_shift, on=index_col, how='left').fillna(0)

In [22]:
all_data.to_csv(data_path+'new_sales_v1_reduced_w_lag.csv',index=False)

In [23]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,item_cat_block_target_sum,item_cat_block_target_mean,item_block_target_mean_lag_1,item_block_target_sum_lag_1,shop_block_target_mean_lag_1,shop_block_target_sum_lag_1,item_cat_block_target_mean_lag_1,item_cat_block_target_sum_lag_1,target_lag_1,item_block_target_mean_lag_2,item_block_target_sum_lag_2,shop_block_target_mean_lag_2,shop_block_target_sum_lag_2,item_cat_block_target_mean_lag_2,item_cat_block_target_sum_lag_2,target_lag_2,item_block_target_mean_lag_3,item_block_target_sum_lag_3,shop_block_target_mean_lag_3,shop_block_target_sum_lag_3,item_cat_block_target_mean_lag_3,item_cat_block_target_sum_lag_3,target_lag_3,item_block_target_mean_lag_5,item_block_target_sum_lag_5,shop_block_target_mean_lag_5,shop_block_target_sum_lag_5,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12
0,59,22154,0,1.0,ЯВЛЕНИЕ 2012 (BD),37,2017.0,1.092041,18.0,1.0,6094.0,1.053414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,59,2552,0,0.0,DEEP PURPLE The House Of Blue Light LP,58,2017.0,1.092041,0.0,0.0,287.0,0.969595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,59,2554,0,0.0,DEEP PURPLE Who Do You Think We Are LP,58,2017.0,1.092041,1.0,1.0,287.0,0.969595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,59,2555,0,0.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,2017.0,1.092041,2.0,1.0,268.0,1.007519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,59,2564,0,0.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,2017.0,1.092041,5.0,1.0,701.0,0.990113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
all_data.shape

(10913804, 47)

In [25]:
all_data = all_data[all_data['date_block_num'] >= 12]

In [39]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,item_cat_block_target_sum,item_cat_block_target_mean,item_block_target_mean_lag_1,item_block_target_sum_lag_1,shop_block_target_mean_lag_1,shop_block_target_sum_lag_1,item_cat_block_target_mean_lag_1,item_cat_block_target_sum_lag_1,target_lag_1,item_block_target_mean_lag_2,item_block_target_sum_lag_2,shop_block_target_mean_lag_2,shop_block_target_sum_lag_2,item_cat_block_target_mean_lag_2,item_cat_block_target_sum_lag_2,target_lag_2,item_block_target_mean_lag_3,item_block_target_sum_lag_3,shop_block_target_mean_lag_3,shop_block_target_sum_lag_3,item_cat_block_target_mean_lag_3,item_cat_block_target_sum_lag_3,target_lag_3,item_block_target_mean_lag_5,item_block_target_sum_lag_5,shop_block_target_mean_lag_5,shop_block_target_sum_lag_5,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor,December
4488710,54,10297,12,4.0,ГАДКИЙ Я 1-2 (BD),37,8198.0,1.295717,23.0,1.000000,7511.0,1.017199,1.000000,42.0,1.363205,10055.0,1.049310,9959.0,3.0,1.000000,2.0,1.413036,7978.0,1.065480,7420.0,0.0,0.000000,0.0,1.337876,6676.0,1.063016,6950.0,0.0,0.0,0.0,1.231158,7792.0,1.051357,8680.0,0.0,0.0,0.0,1.247143,9386.0,1.053414,6094.0,0.0,1,0,0,0,0
4488711,54,10296,12,3.0,ГАДКИЙ Я 1-2 (3D BD),38,8198.0,1.295717,17.0,1.000000,1410.0,1.002132,1.000000,24.0,1.363205,10055.0,1.018868,2052.0,0.0,0.000000,0.0,1.413036,7978.0,1.038710,1449.0,0.0,0.000000,0.0,1.337876,6676.0,1.048142,1241.0,0.0,0.0,0.0,1.231158,7792.0,1.023669,1384.0,0.0,0.0,0.0,1.247143,9386.0,1.019387,1630.0,0.0,1,0,0,0,0
4488712,54,10298,12,14.0,ГАДКИЙ Я 2,40,8198.0,1.295717,182.0,1.181818,22065.0,1.105627,1.308511,369.0,1.363205,10055.0,1.163135,28598.0,21.0,2.226191,1309.0,1.413036,7978.0,1.187626,25433.0,119.0,2.526316,144.0,1.337876,6676.0,1.168003,24173.0,7.0,0.0,0.0,1.231158,7792.0,1.102380,24130.0,0.0,0.0,0.0,1.247143,9386.0,1.145903,33489.0,0.0,1,0,0,0,0
4488713,54,10300,12,3.0,ГАДКИЙ Я 2 (BD),37,8198.0,1.295717,26.0,0.962963,7511.0,1.017199,1.058824,54.0,1.363205,10055.0,1.049310,9959.0,1.0,1.367424,361.0,1.413036,7978.0,1.065480,7420.0,31.0,1.606061,53.0,1.337876,6676.0,1.063016,6950.0,0.0,0.0,0.0,1.231158,7792.0,1.051357,8680.0,0.0,0.0,0.0,1.247143,9386.0,1.053414,6094.0,0.0,1,0,0,0,0
4488714,54,10284,12,1.0,"Высоцкий Владимир Спасибо, что живой (mp3-CD)...",57,8198.0,1.295717,3.0,1.000000,984.0,1.004082,1.000000,4.0,1.363205,10055.0,1.016405,1425.0,0.0,1.000000,3.0,1.413036,7978.0,1.009579,1054.0,0.0,1.000000,5.0,1.337876,6676.0,1.009346,1080.0,0.0,1.0,10.0,1.231158,7792.0,1.005348,1316.0,0.0,0.0,0.0,1.247143,9386.0,1.003914,1539.0,0.0,1,0,0,0,0


In [28]:
 all_data['date_block_num'].unique()

array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31, 32, 33], dtype=int64)

# Calulating the Holiday Month features

In [34]:
all_data['Newyear_Xmas'] = all_data.date_block_num.apply(lambda x: 1 if x in [12,24] else 0)
all_data['Valentine_MenDay'] = all_data.date_block_num.apply(lambda x: 1 if x in [13,25] else 0)
all_data['WomenDay'] = all_data.date_block_num.apply(lambda x: 1 if x in [14,26] else 0)
all_data['Easter_Labor'] = all_data.date_block_num.apply(lambda x: 1 if x in [15,27] else 0)
all_data['December'] = all_data.date_block_num.apply(lambda x: 1 if x ==23 else 0)

In [36]:
all_data.to_pickle(data_path+'new_sales_lag_after12.pickle')

In [37]:
all_data = pd.read_pickle(data_path+'new_sales_lag_after12.pickle')

In [41]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_sum,item_block_target_mean,item_cat_block_target_sum,item_cat_block_target_mean,item_block_target_mean_lag_1,item_block_target_sum_lag_1,shop_block_target_mean_lag_1,shop_block_target_sum_lag_1,item_cat_block_target_mean_lag_1,item_cat_block_target_sum_lag_1,target_lag_1,item_block_target_mean_lag_2,item_block_target_sum_lag_2,shop_block_target_mean_lag_2,shop_block_target_sum_lag_2,item_cat_block_target_mean_lag_2,item_cat_block_target_sum_lag_2,target_lag_2,item_block_target_mean_lag_3,item_block_target_sum_lag_3,shop_block_target_mean_lag_3,shop_block_target_sum_lag_3,item_cat_block_target_mean_lag_3,item_cat_block_target_sum_lag_3,target_lag_3,item_block_target_mean_lag_5,item_block_target_sum_lag_5,shop_block_target_mean_lag_5,shop_block_target_sum_lag_5,item_cat_block_target_mean_lag_5,item_cat_block_target_sum_lag_5,target_lag_5,item_block_target_mean_lag_12,item_block_target_sum_lag_12,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor,December
4488710,54,10297,12,4.0,ГАДКИЙ Я 1-2 (BD),37,8198.0,1.295717,23.0,1.000000,7511.0,1.017199,1.000000,42.0,1.363205,10055.0,1.049310,9959.0,3.0,1.000000,2.0,1.413036,7978.0,1.065480,7420.0,0.0,0.000000,0.0,1.337876,6676.0,1.063016,6950.0,0.0,0.0,0.0,1.231158,7792.0,1.051357,8680.0,0.0,0.0,0.0,1.247143,9386.0,1.053414,6094.0,0.0,1,0,0,0,0
4488711,54,10296,12,3.0,ГАДКИЙ Я 1-2 (3D BD),38,8198.0,1.295717,17.0,1.000000,1410.0,1.002132,1.000000,24.0,1.363205,10055.0,1.018868,2052.0,0.0,0.000000,0.0,1.413036,7978.0,1.038710,1449.0,0.0,0.000000,0.0,1.337876,6676.0,1.048142,1241.0,0.0,0.0,0.0,1.231158,7792.0,1.023669,1384.0,0.0,0.0,0.0,1.247143,9386.0,1.019387,1630.0,0.0,1,0,0,0,0
4488712,54,10298,12,14.0,ГАДКИЙ Я 2,40,8198.0,1.295717,182.0,1.181818,22065.0,1.105627,1.308511,369.0,1.363205,10055.0,1.163135,28598.0,21.0,2.226191,1309.0,1.413036,7978.0,1.187626,25433.0,119.0,2.526316,144.0,1.337876,6676.0,1.168003,24173.0,7.0,0.0,0.0,1.231158,7792.0,1.102380,24130.0,0.0,0.0,0.0,1.247143,9386.0,1.145903,33489.0,0.0,1,0,0,0,0
4488713,54,10300,12,3.0,ГАДКИЙ Я 2 (BD),37,8198.0,1.295717,26.0,0.962963,7511.0,1.017199,1.058824,54.0,1.363205,10055.0,1.049310,9959.0,1.0,1.367424,361.0,1.413036,7978.0,1.065480,7420.0,31.0,1.606061,53.0,1.337876,6676.0,1.063016,6950.0,0.0,0.0,0.0,1.231158,7792.0,1.051357,8680.0,0.0,0.0,0.0,1.247143,9386.0,1.053414,6094.0,0.0,1,0,0,0,0
4488714,54,10284,12,1.0,"Высоцкий Владимир Спасибо, что живой (mp3-CD)...",57,8198.0,1.295717,3.0,1.000000,984.0,1.004082,1.000000,4.0,1.363205,10055.0,1.016405,1425.0,0.0,1.000000,3.0,1.413036,7978.0,1.009579,1054.0,0.0,1.000000,5.0,1.337876,6676.0,1.009346,1080.0,0.0,1.0,10.0,1.231158,7792.0,1.005348,1316.0,0.0,0.0,0.0,1.247143,9386.0,1.003914,1539.0,0.0,1,0,0,0,0


In [40]:
all_data.shape

(6425094, 52)